In [38]:
import urllib.request
import re
from bs4 import BeautifulSoup
#import sqlite3
import time
x = input('輸入X軸座標:')
y = input('輸入X軸座標:')
page = 1
coordinate_x = []
coordinate_y = []
crops_type = []
crop_oases = []

for i in range(1,10):
    
    if page <= 1:
        url = 'http://croppers.travibot.com/?l=en&server=tx2.travian.com&x={}&y={}'.format(x,y)
    else:
        url = 'http://croppers.travibot.com/?l=en&server=tx2.travian.com&x={}&y={}&page={}'.format(x,y,page)
    print ("searching: {}".format(url))
    response = urllib.request.urlopen(url)
    html = response.read()
    sp = BeautifulSoup(html)
    temp1 = sp.find_all("td",class_="l")
    temp2 = sp.find_all("a",href=re.compile("http://tx2.travian.com/karte.php?"))

    for i in temp1:
        if i.find_all("img",alt=re.compile("Cropper type")):
            crops_type.append(i.get_text())
        elif i.find_all("img",src=re.compile("/img/4.gif")):
            crop_oases.append('Yes')
        else:
            crop_oases.append('No')

    for i in temp2:
        coordinate_x.append(i.get_text().split('|')[0])
        coordinate_y.append(i.get_text().split('|')[1])
    
    page+=1
    time.sleep(3)
    
count = 0
show = ()
for i in coordinate_x:
    show = (coordinate_x[count],coordinate_y[count],crops_type[count],crop_oases[count])
    count+=1
    print(show)
    
#connect_db = sqlite3.connect('travian1.db')
#c = connect_db.cursor()
#c.execute('''DROP TABLE crops_info''')
#c.execute('''CREATE TABLE crops_info (coordinate_x real, coordinate_y real, crops_type real, crop_oases text)''')

#count = 0
#for i in coordinate_x:
#    add_info = (coordinate_x[count],coordinate_y[count],crops_type[count],crop_oases[count])
#    c.execute('INSERT INTO crops_info VALUES (?,?,?,?)', add_info)
#    count+=1

#c.execute('''SELECT * FROM crops_info''')

#connect_db.commit()
#connect_db.close()

輸入X軸座標:1
輸入X軸座標:1
searching: http://croppers.travibot.com/?l=en&server=tx2.travian.com&x=1&y=1
searching: http://croppers.travibot.com/?l=en&server=tx2.travian.com&x=1&y=1&page=2
('0', '0', ' 9', 'No')
('2', '3', ' 9', 'No')
('2', '5', ' 15', 'Yes')
('6', '2', ' 15', 'No')
('3', '6', ' 15', 'Yes')
('-3', '-3', ' 15', 'No')
('4', '6', ' 9', 'Yes')
('6', '-2', ' 15', 'Yes')
('-4', '-3', ' 15', 'No')
('5', '6', ' 15', 'No')
('-2', '-6', ' 15', 'No')
('8', '-2', ' 9', 'Yes')
('-5', '-4', ' 15', 'No')
('2', '-7', ' 9', 'No')
('2', '9', ' 15', 'Yes')
('0', '9', ' 15', 'Yes')
